# 自动站数据的查询




In [44]:
%time
import numpy as np
import pandas as pd
import datetime as dtt
from datetime import *

'''
1.计算期间的降水
2.查找降水超过2mm的站点 --
3.计算按钮 --- 
4.设计按钮的通用数据处理方式

'''

class zdz_data:
    def __init__(self, start, end):
        self.start = start
        self.end = end
    def sql_all(self):
        pass
    def sql_day(self):
        pass
    def read_csv(self):
        '''
        1.数据读取---此方法为测试方法
        2.数据分段---每天
        3.每天统计
        '''
#         station_Mws = pd.read_csv("Mws_15.csv")
#         station_Aws = pd.read_csv("Aws_15.csv")
#         station_all = pd.concat([station_Aws, station_Mws])
        # station_all = pd.read_csv("data_zdz_height.csv")
        station_all = pd.read_csv("data_zdz_height.csv")
        return station_all
    def return_daylist(self):
        '''
        返回日期列表
        '''
        dates = []
        #         dt = datetime.datetime.strptime(self.start, "%Y-%m-%d")
        dt = dtt.datetime.strptime(self.start[0:10], "%Y-%m-%d")
        date = self.start[:10]
        yesday = dt+dtt.timedelta(-1)
        dates.append(yesday.strftime("%Y-%m-%d")[:10])
        while date <= self.end[:10]:
            dates.append(date)
            dt = dt + dtt.timedelta(1)
            date = dt.strftime("%Y-%m-%d")
        return dates
    def day_button(self,city):
        '''返回按钮组'''
        station_all = self.read_csv()
        dates = self.return_daylist()
        dailylist = []
        for i in range(len(dates)):
            daily = {
                "time":None,
                "rain":False,
                "wind":False,
                "tman":False,
                "tmin":False,
                "view":False,
            }
            if i!=0:
                s_date = dates[i-1] + ' ' + '20:00'
                e_date = dates[i] + ' ' + '20:00'
                daily["time"] = dates[i]
                df_preday = station_all[(station_all['tTime'] >= s_date) & (station_all['tTime'] <= e_date)]
                # 统计标量
                temp_min = df_preday[(df_preday['Height'] < 6000) & (df_preday['T'] > -400)]['T'].min()
                temp_max = df_preday[(df_preday['Height'] < 6000) & (df_preday['T'] > -400)]['T'].max()
                vv = df_preday[(df_preday['VV'] > 0.1) & (df_preday['VV'] < 500)]['VV'].count()
                RR = df_preday[(df_preday['RR'] > 0.1) & (df_preday['RR'] < 8888)]['RR'].count()
                wind = df_preday[df_preday['fFy'] > 187]['fFy'].count()    
                if temp_min<0:
                    daily["tmin"] = True
                if temp_max>350:
                    daily["tmax"] = True
                if vv >0:
                    daily["view"] = True
                if RR >0:
                    daily["rain"] = True
                if wind >0:
                    daily["wind"] = True
                dailylist.append(daily)
        return dailylist
    def rain_data(self,start,end):
        '''返回点击绘图所需的数据'''
        # 采集数据 可以是sql
        station_all = self.read_csv()
        grouped_tTime = station_all.groupby('IIiii')
        points = []
        table_data = []  
        for i in grouped_tTime.size().index:
            data = grouped_tTime.get_group(i)
            RR = data[(data['RR'] > 0.0) & (data['RR'] < 8888)]['RR'].sum()/10.0
            if RR > 0:
                single = {
                    "type": "Feature",
                    "properties": {
                        "value": str(RR)
                    },
                    "geometry": {
                        "type": "Point",
                        "coordinates": [data['lon'].iloc[0], data['lat'].iloc[0]]
                    }
                }
                points.append(single)  
                # 采集marker和表格的数据
                rain = data.sort_values(by="tTime")["RR"].to_list()
                time = data.sort_values(by="tTime")["tTime"].to_list()
                ## 自己定义表格的数据形式
                single_rain = {
                    "IIiii":data['IIiii'].iloc[0],
                    "county":data['county'].iloc[0],
                    "town":data['Town'].iloc[0],
                    "StationName":data['StationName'].iloc[0],
                    "RR":RR,
                    "lat":data['lat'].iloc[0],
                    "lon":data['lon'].iloc[0],
                    "height":data['Height'].iloc[0],
                    "rain_list":rain,
                    "time_list":time
                }
                table_data.append(single_rain)  
        return points
    def index_data(self):
        '''返回所有数据'''
        city = "taizhou"
        daily_btn_list = self.day_button(city)
        station_all = self.read_csv()
        grouped_tTime = station_all.groupby('IIiii')
        table_data = []  
        points = []
        for i in grouped_tTime.size().index:
            data = grouped_tTime.get_group(i)
            RR = data[(data['RR'] > 0.0) & (data['RR'] < 8888)]['RR'].sum()/10.0
            tmin = data[(data['T'] >-400) & (data['T'] < 8888)]['T'].min()/10.0
            tmax = data[(data['T'] >-400) & (data['T'] < 8888)]['T'].max()/10.0
            vv = data[(data['VV'] >0) & (data['VV'] < 1000)]['VV'].min()
            wind = data[data['fFy'] > 187]['fFy'].max()/10.0
            rain_data = {
                "type": "Feature",
                "properties": {
                    "value": str(RR)
                },
                "geometry": {
                    "type": "Point",
                    "coordinates": [data['lon'].iloc[0], data['lat'].iloc[0]]
                }
            }
            points.append(rain_data)
            rain = data.sort_values(by="tTime")["RR"].to_list()
            time = data.sort_values(by="tTime")["tTime"].to_list()
            single = {
                "IIiii":data['IIiii'].iloc[0],
                "county":data['county'].iloc[0],
                "town":data['Town'].iloc[0],
                "StationName":data['StationName'].iloc[0],
                "fFy":wind,
                "RR":RR,
                "Tx":tmax,
                "Tn":tmin,
                "VV":vv,
                "lat":data['lat'].iloc[0],
                "lon":data['lon'].iloc[0],
                "rain_list":rain,
                "time_list":time
                }
            table_data.append(single)
        return table_data,points,daily_btn_list

            
                     
                
                
start = '2022-01-25 20:00'
end = '2022-02-10 06:00'
city = "taizhou"

zdz_worker = zdz_data(start, end)

# zdz_worker.day_button(city)
# points = zdz_worker.rain_data()
table_data,points = zdz_worker.index_data()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 27.9 µs


# 自动站数据的查询

In [ ]:
import pymysql
import numpy as np
import pandas as pd

In [1]:
import pymysql
import numpy as np
import pandas as pd
conn = pymysql.connect(host="127.0.0.1",port=3306,user="root",passwd="051219",db="ZJSZDZDB")
# 2023-05-11 20:00:00 2023-05-11 21:11:00 2023-05-10 21:11:00 2023-05-11 20:00:00

start2 = '2019-08-07 21:11:00'
end2 = '2019-08-08 20:00:00'

start1 = '2019-08-08 20:00:00'
end1 =  '2019-08-08 21:11:00'

sql1 = '''select tTime,Ri from Tab_AM_M where (tTime between '{start}' and '{end}' and IIiii = '58653') order by tTime'''            
rsql1 = sql1.format(start=start1,end=end1) 
rsql2 = sql1.format(start=start2,end=end2) 
df1 = pd.read_sql(rsql1 , con=conn)              
df2 = pd.read_sql(rsql2 , con=conn)  

df2 = df2[:-1]

/opt/conda/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
station_all = pd.concat([df1, df2])

In [3]:
station_all['tTime'].values

array(['2019-08-08 20:00:00', '2019-08-08 20:00:00',
       '2019-08-08 20:05:00', '2019-08-08 20:10:00',
       '2019-08-08 20:15:00', '2019-08-08 20:20:00',
       '2019-08-08 20:25:00', '2019-08-08 20:30:00',
       '2019-08-08 20:35:00', '2019-08-08 20:40:00',
       '2019-08-08 20:45:00', '2019-08-08 20:50:00',
       '2019-08-08 20:55:00', '2019-08-08 21:00:00',
       '2019-08-08 21:05:00', '2019-08-08 21:10:00',
       '2019-08-07 21:15:00', '2019-08-07 21:20:00',
       '2019-08-07 21:25:00', '2019-08-07 21:30:00',
       '2019-08-07 21:35:00', '2019-08-07 21:40:00',
       '2019-08-07 21:45:00', '2019-08-07 21:50:00',
       '2019-08-07 21:55:00', '2019-08-07 22:00:00',
       '2019-08-07 22:05:00', '2019-08-07 22:10:00',
       '2019-08-07 22:15:00', '2019-08-07 22:20:00',
       '2019-08-07 22:25:00', '2019-08-07 22:30:00',
       '2019-08-07 22:35:00', '2019-08-07 22:40:00',
       '2019-08-07 22:45:00', '2019-08-07 22:50:00',
       '2019-08-07 22:55:00', '2019-08-07 23:0

In [6]:
df1

,tTime,Ri
0,2019-08-08 20:00:00,1.0
1,2019-08-08 20:00:00,1.0
2,2019-08-08 20:05:00,0.0
3,2019-08-08 20:10:00,0.0
4,2019-08-08 20:15:00,1.0
5,2019-08-08 20:20:00,0.0
6,2019-08-08 20:25:00,0.0
7,2019-08-08 20:30:00,1.0
8,2019-08-08 20:35:00,0.0
9,2019-08-08 20:40:00,1.0


In [7]:
df2

,tTime,Ri
0,2019-08-07 21:15:00,0.0
1,2019-08-07 21:20:00,0.0
2,2019-08-07 21:25:00,0.0
3,2019-08-07 21:30:00,0.0
4,2019-08-07 21:35:00,0.0
...,...,...
269,2019-08-08 19:40:00,0.0
270,2019-08-08 19:45:00,0.0
271,2019-08-08 19:50:00,0.0
272,2019-08-08 19:55:00,0.0


In [19]:
import pymysql
import numpy as np
import pandas as pd
conn = pymysql.connect(host="127.0.0.1",port=3306,user="root",passwd="051219",db="ZJSZDZDB")
# 2023-05-11 20:00:00 2023-05-11 21:11:00 2023-05-10 21:11:00 2023-05-11 20:00:00



start1 = '2019-08-07 20:00:00'
end1 =  '2019-08-08 20:00:00'

sql1 = '''select tTime,Ri from Tab_AM_M where (tTime between '{start}' and '{end}' and IIiii = '58653') order by tTime'''            
rsql1 = sql1.format(start=start1,end=end1) 

df1 = pd.read_sql(rsql1 , con=conn)              
df1

,tTime,Ri
0,2019-08-07 20:00:00,0.0
1,2019-08-07 20:05:00,0.0
2,2019-08-07 20:10:00,0.0
3,2019-08-07 20:15:00,0.0
4,2019-08-07 20:20:00,0.0
...,...,...
285,2019-08-08 19:45:00,0.0
286,2019-08-08 19:50:00,0.0
287,2019-08-08 19:55:00,0.0
288,2019-08-08 20:00:00,1.0


In [ ]:
20 21 22 23 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20

20 21 


In [ ]:
2019-08-08 20:00:00

In [3]:
import pymysql
import numpy as np
import pandas as pd
conn = pymysql.connect(host="127.0.0.1",port=3306,user="root",passwd="051219",db="ZJSZDZDB")
# 2023-05-11 20:00:00 2023-05-11 21:11:00 2023-05-10 21:11:00 2023-05-11 20:00:00



start1 = '2019-08-07 20:00:00'
end1 =  '2019-08-08 20:00:00'

sql1 = '''select tTime,Ri from Tab_AM_M where (tTime = '2019-08-08 20:00:00' and IIiii = '58653') order by tTime'''            
rsql1 = sql1.format(start=start1,end=end1) 

df1 = pd.read_sql(sql1 , con=conn)              
df1 = df1[1:]

AttributeError: 'DataFrame' object has no attribute 'dorp'

In [ ]:
start2 = '2019-08-07 11:11:00'
end2 = '2019-08-07 20:00:00'

start1 = '2019-08-07 20:00:00'
end1 =  '2019-08-08 11:11:00'